In [1]:
import sqlite3
import json

In [2]:
con = sqlite3.connect("db_20191123-20221018.sqlite")
con.execute("ATTACH 'db_20200825-20211226.sqlite' as db_2")
con.execute("ATTACH 'db_20220104-20230518.sqlite' as db_3")
con.row_factory = sqlite3.Row
cur = con.cursor()


In [3]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cur.fetchall())
cur.execute("SELECT name FROM db_2.sqlite_master")
print(cur.fetchall())
cur.execute("SELECT name FROM db_3.sqlite_master")
print(cur.fetchall())

[<sqlite3.Row object at 0x0000025FBE3FB7C0>, <sqlite3.Row object at 0x0000025FBE3FBB50>, <sqlite3.Row object at 0x0000025FBE3F9000>, <sqlite3.Row object at 0x0000025FBE3FBC70>, <sqlite3.Row object at 0x0000025FBE3FB8E0>, <sqlite3.Row object at 0x0000025FBE3FB760>, <sqlite3.Row object at 0x0000025FBE3FA410>, <sqlite3.Row object at 0x0000025FBE3F9090>, <sqlite3.Row object at 0x0000025FBE3FB1F0>, <sqlite3.Row object at 0x0000025FBE3FB580>, <sqlite3.Row object at 0x0000025FBE3F90C0>, <sqlite3.Row object at 0x0000025FBE3FB220>, <sqlite3.Row object at 0x0000025FBE3FBFD0>, <sqlite3.Row object at 0x0000025FBE3FBBB0>, <sqlite3.Row object at 0x0000025FBE3FB700>, <sqlite3.Row object at 0x0000025FBE3FBC10>, <sqlite3.Row object at 0x0000025FBE3F8670>, <sqlite3.Row object at 0x0000025FBE3FAE00>, <sqlite3.Row object at 0x0000025FBE3F8190>, <sqlite3.Row object at 0x0000025FBE3F9CC0>, <sqlite3.Row object at 0x0000025FBE3F82B0>, <sqlite3.Row object at 0x0000025FBE3FB3D0>, <sqlite3.Row object at 0x000002

In [34]:
query = """
    SELECT * from messages, db_2.messages, db_3.messages
    """
cur.execute(query)
print(cur.fetchone().keys())

['rowid', 'id', 'json', 'readStatus', 'expires_at', 'sent_at', 'schemaVersion', 'conversationId', 'received_at', 'source', 'hasAttachments', 'hasFileAttachments', 'hasVisualMediaAttachments', 'expireTimer', 'expirationStartTimestamp', 'type', 'body', 'messageTimer', 'messageTimerStart', 'messageTimerExpiresAt', 'isErased', 'isViewOnce', 'sourceUuid', 'serverGuid', 'sourceDevice', 'storyId', 'isStory', 'isChangeCreatedByUs', 'isTimerChangeFromSync', 'isGroupLeaveEvent', 'isGroupLeaveEventFromOther', 'seenStatus', 'storyDistributionListId', 'callId', 'callMode', 'expiresAt', 'shouldAffectActivity', 'shouldAffectPreview', 'isUserInitiatedMessage']


In [42]:
query = """
    SELECT 'messages' AS table_name, COUNT(*) FROM messages
    UNION
    SELECT 'db_2.messages' AS table_name, COUNT(*) FROM db_2.messages
    UNION
    SELECT 'db_3.messages' AS table_name, COUNT(*) FROM db_3.messages
"""

cur.execute(query)
sum = 0
for count in cur.fetchall():
    sum += count['COUNT(*)']
print(str(sum) + " total messages.")

query = """
    SELECT COUNT(*) FROM (
        SELECT sent_at FROM messages
        UNION
        SELECT sent_at FROM db_2.messages
        UNION
        SELECT sent_at FROM db_3.messages
    ) AS derived;
"""

cur.execute(query)
unique = cur.fetchone()['COUNT(*)']
print(str(unique) + " unique messages.")

109222 total messages.
84340 unique messages.


In [51]:
query = """
    WITH combined_messages AS (
        SELECT sent_at, rowid, id, json, readStatus, conversationId, received_at, source, hasAttachments, hasFileAttachments, hasVisualMediaAttachments, type, body, isChangeCreatedByUs, isUserInitiatedMessage
        FROM messages
        UNION
        SELECT sent_at, rowid, id, json, readStatus, conversationId, received_at, source, hasAttachments, hasFileAttachments, hasVisualMediaAttachments, type, body, NULL as isChangeCreatedByUs, NULL as isUserInitiatedMessage
        FROM db_2.messages
        UNION
        SELECT sent_at, rowid, id, json, readStatus, conversationId, received_at, source, hasAttachments, hasFileAttachments, hasVisualMediaAttachments, type, body, isChangeCreatedByUs, isUserInitiatedMessage
        FROM db_3.messages
    )
    SELECT *
    FROM (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY sent_at ORDER BY sent_at) AS row_num
        FROM combined_messages
    ) AS numbered_messages
    WHERE conversationId IN (
        SELECT conversationId
        FROM conversations
        WHERE type = 'group'
        UNION
        SELECT conversationId
        FROM db_2.conversations
        WHERE type = 'group'
        UNION
        SELECT conversationId
        FROM db_3.conversations
        WHERE type = 'group'
    )
    AND type IN ('incoming', 'outgoing')
    AND row_num = 1
    ORDER BY sent_at;
"""

cur.execute(query)
for i in range(10):
    row = cur.fetchone()
    display(dict(row))

{'sent_at': 1574559155643,
 'rowid': 3,
 'id': 'f3c50539-9e20-4d17-8a30-28eff3463cca',
 'json': '{"attachments":[],"body":"kss?","contact":[],"conversationId":"ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46","decrypted_at":1574559156496,"delivered":2,"delivered_to":["+12505752707","+12503177584"],"errors":[],"expirationStartTimestamp":1574559155643,"flags":0,"hasAttachments":0,"id":"f3c50539-9e20-4d17-8a30-28eff3463cca","isViewOnce":false,"preview":[],"quote":null,"read_by":["+12505752707","+12503177584"],"received_at":1574559156493,"recipients":["+12503177584","+12505752707"],"requiredProtocolVersion":0,"schemaVersion":10,"sent":true,"sent_at":1574559155643,"sent_to":["+12503177584","+12505752707"],"source":"+12508631889","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559156493,"type":"outgoing","unidentifiedDeliveries":["+12503177584"],"readStatus":0,"seenStatus":2}',
 'readStatus': 0,
 'conversationId': 'ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46',
 'received_at': 157

{'sent_at': 1574559167727,
 'rowid': 4,
 'id': 'f7ac76db-094e-470e-b7ec-2c225d6fc4f8',
 'json': '{"attachments":[],"body":"wentworth?","contact":[],"conversationId":"ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46","decrypted_at":1574559168288,"delivered":2,"delivered_to":["+12505752707","+12503177584"],"errors":[],"expirationStartTimestamp":1574559167727,"flags":0,"hasAttachments":0,"id":"f7ac76db-094e-470e-b7ec-2c225d6fc4f8","isViewOnce":false,"preview":[],"quote":null,"read_by":["+12505752707","+12503177584"],"received_at":1574559168286,"recipients":["+12503177584","+12505752707"],"requiredProtocolVersion":0,"schemaVersion":10,"sent":true,"sent_at":1574559167727,"sent_to":["+12503177584","+12505752707"],"source":"+12508631889","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559168286,"type":"outgoing","unidentifiedDeliveries":["+12503177584"],"readStatus":0,"seenStatus":2}',
 'readStatus': 0,
 'conversationId': 'ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46',
 'received_at

{'sent_at': 1574559518838,
 'rowid': 5,
 'id': '7e7361d1-529c-49b5-b250-677a6236cd22',
 'json': '{"attachments":[],"body":"That’ll satisfy me","contact":[],"conversationId":"ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46","decrypted_at":1574559519457,"errors":[],"flags":0,"hasAttachments":0,"id":"7e7361d1-529c-49b5-b250-677a6236cd22","isViewOnce":false,"preview":[],"quote":null,"received_at":1574559519456,"requiredProtocolVersion":0,"schemaVersion":10,"sent_at":1574559518838,"source":"+12505752707","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559519456,"type":"incoming","unidentifiedDeliveryReceived":true,"readStatus":0,"seenStatus":0}',
 'readStatus': 0,
 'conversationId': 'ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46',
 'received_at': 1574559519456,
 'source': 12505752707,
 'hasAttachments': 0,
 'hasFileAttachments': None,
 'hasVisualMediaAttachments': None,
 'type': 'incoming',
 'body': 'That’ll satisfy me',
 'isChangeCreatedByUs': 0,
 'isUserInitiatedMessage': 1,
 'r

{'sent_at': 1574559520701,
 'rowid': 6,
 'id': '06db70d9-0d8a-4725-acea-ed6b4ce1c7e3',
 'json': '{"attachments":[],"body":"hey erik!","contact":[],"conversationId":"ae62bcd9-bd74-4a72-9426-e31dfb761fb9","decrypted_at":1574559521804,"delivered":1,"delivered_to":["+17783637575"],"errors":[],"expirationStartTimestamp":1574559520701,"flags":0,"hasAttachments":0,"id":"06db70d9-0d8a-4725-acea-ed6b4ce1c7e3","isViewOnce":false,"preview":[],"quote":null,"read_by":["+17783637575"],"received_at":1574559521796,"recipients":["+17783637575"],"requiredProtocolVersion":0,"schemaVersion":10,"sent":true,"sent_at":1574559520701,"sent_to":["+17783637575"],"source":"+12508631889","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559521796,"type":"outgoing","unidentifiedDeliveries":[],"readStatus":0,"seenStatus":2}',
 'readStatus': 0,
 'conversationId': 'ae62bcd9-bd74-4a72-9426-e31dfb761fb9',
 'received_at': 1574559521796,
 'source': 12508631889,
 'hasAttachments': 0,
 'hasFileAt

{'sent_at': 1574559526921,
 'rowid': 7,
 'id': '9a70a39f-ac69-41ff-bc53-48ba0d135507',
 'json': '{"attachments":[],"body":"Ben what’re you allergic to","contact":[],"conversationId":"ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46","decrypted_at":1574559527505,"errors":[],"flags":0,"hasAttachments":0,"id":"9a70a39f-ac69-41ff-bc53-48ba0d135507","isViewOnce":false,"preview":[],"quote":null,"received_at":1574559527505,"requiredProtocolVersion":0,"schemaVersion":10,"sent_at":1574559526921,"source":"+12505752707","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559527505,"type":"incoming","unidentifiedDeliveryReceived":true,"readStatus":0,"seenStatus":0}',
 'readStatus': 0,
 'conversationId': 'ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46',
 'received_at': 1574559527505,
 'source': 12505752707,
 'hasAttachments': 0,
 'hasFileAttachments': None,
 'hasVisualMediaAttachments': None,
 'type': 'incoming',
 'body': 'Ben what’re you allergic to',
 'isChangeCreatedByUs': 0,
 'isUserInitiat

{'sent_at': 1574559574014,
 'rowid': 9,
 'id': 'd77468dd-1e6f-4c79-b8a7-0cb35c30f048',
 'json': '{"attachments":[],"body":"oh for verification","contact":[],"conversationId":"ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46","decrypted_at":1574559574630,"delivered":2,"delivered_to":["+12505752707","+12503177584"],"errors":[],"expirationStartTimestamp":1574559574014,"flags":0,"hasAttachments":0,"id":"d77468dd-1e6f-4c79-b8a7-0cb35c30f048","isViewOnce":false,"preview":[],"quote":null,"read_by":["+12505752707","+12503177584"],"received_at":1574559574628,"recipients":["+12503177584","+12505752707"],"requiredProtocolVersion":0,"schemaVersion":10,"sent":true,"sent_at":1574559574014,"sent_to":["+12503177584","+12505752707"],"source":"+12508631889","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559574628,"type":"outgoing","unidentifiedDeliveries":["+12503177584"],"readStatus":0,"seenStatus":2}',
 'readStatus': 0,
 'conversationId': 'ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46',
 're

{'sent_at': 1574559590622,
 'rowid': 10,
 'id': '7b0e7504-8402-45e6-86d8-78c649dd53b3',
 'json': '{"attachments":[],"body":"Yeah","contact":[],"conversationId":"ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46","decrypted_at":1574559591213,"errors":[],"flags":0,"hasAttachments":0,"id":"7b0e7504-8402-45e6-86d8-78c649dd53b3","isViewOnce":false,"preview":[],"quote":null,"received_at":1574559591212,"requiredProtocolVersion":0,"schemaVersion":10,"sent_at":1574559590622,"source":"+12505752707","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559591212,"type":"incoming","unidentifiedDeliveryReceived":true,"readStatus":0,"seenStatus":0}',
 'readStatus': 0,
 'conversationId': 'ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46',
 'received_at': 1574559591212,
 'source': 12505752707,
 'hasAttachments': 0,
 'hasFileAttachments': None,
 'hasVisualMediaAttachments': None,
 'type': 'incoming',
 'body': 'Yeah',
 'isChangeCreatedByUs': 0,
 'isUserInitiatedMessage': 1,
 'row_num': 1}

{'sent_at': 1574559599171,
 'rowid': 11,
 'id': '30fe8466-dcab-42e2-86e2-69ea52f51d90',
 'json': '{"attachments":[],"body":"Can’t be too careful these days yknow","contact":[],"conversationId":"ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46","decrypted_at":1574559599770,"errors":[],"flags":0,"hasAttachments":0,"id":"30fe8466-dcab-42e2-86e2-69ea52f51d90","isViewOnce":false,"preview":[],"quote":null,"received_at":1574559599769,"requiredProtocolVersion":0,"schemaVersion":10,"sent_at":1574559599171,"source":"+12505752707","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559599769,"type":"incoming","unidentifiedDeliveryReceived":true,"readStatus":0,"seenStatus":0}',
 'readStatus': 0,
 'conversationId': 'ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46',
 'received_at': 1574559599769,
 'source': 12505752707,
 'hasAttachments': 0,
 'hasFileAttachments': None,
 'hasVisualMediaAttachments': None,
 'type': 'incoming',
 'body': 'Can’t be too careful these days yknow',
 'isChangeCreatedByUs

{'sent_at': 1574559629977,
 'rowid': 12,
 'id': 'dcfb0b48-944a-42c9-980a-d9e6ca0d4eb5',
 'json': '{"attachments":[],"body":"true","contact":[],"conversationId":"ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46","decrypted_at":1574559630608,"delivered":2,"delivered_to":["+12505752707","+12503177584"],"errors":[],"expirationStartTimestamp":1574559629977,"flags":0,"hasAttachments":0,"id":"dcfb0b48-944a-42c9-980a-d9e6ca0d4eb5","isViewOnce":false,"preview":[],"quote":null,"read_by":["+12505752707","+12503177584"],"received_at":1574559630606,"recipients":["+12503177584","+12505752707"],"requiredProtocolVersion":0,"schemaVersion":10,"sent":true,"sent_at":1574559629977,"sent_to":["+12503177584","+12505752707"],"source":"+12508631889","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574559630606,"type":"outgoing","unidentifiedDeliveries":["+12503177584"],"readStatus":0,"seenStatus":2}',
 'readStatus': 0,
 'conversationId': 'ba0a5b7d-02be-46e3-8f47-18ccbe8a1b46',
 'received_at': 15

{'sent_at': 1574560069504,
 'rowid': 13,
 'id': '61ecf3c8-7196-40d6-b394-1f2c59f27832',
 'json': '{"attachments":[],"body":"hello dan (secured message)","contact":[],"conversationId":"ae62bcd9-bd74-4a72-9426-e31dfb761fb9","decrypted_at":1574560071525,"errors":[],"flags":0,"hasAttachments":0,"id":"61ecf3c8-7196-40d6-b394-1f2c59f27832","isViewOnce":false,"preview":[],"quote":null,"received_at":1574560071524,"requiredProtocolVersion":0,"schemaVersion":10,"sent_at":1574560069504,"source":"+17783637575","sourceDevice":1,"sticker":null,"supportedVersionAtReceive":3,"timestamp":1574560071524,"type":"incoming","unidentifiedDeliveryReceived":true,"readStatus":0,"seenStatus":0}',
 'readStatus': 0,
 'conversationId': 'ae62bcd9-bd74-4a72-9426-e31dfb761fb9',
 'received_at': 1574560071524,
 'source': 17783637575,
 'hasAttachments': 0,
 'hasFileAttachments': None,
 'hasVisualMediaAttachments': None,
 'type': 'incoming',
 'body': 'hello dan (secured message)',
 'isChangeCreatedByUs': 0,
 'isUserInitia